In [1]:
import glob
import pickle
import random
import re
import sys
from typing import List

import ase
import ase.io
import numpy as np
from tqdm import tqdm

import jax
import jax.nn
import jax.numpy as jnp

In [2]:
with open('PBE_DFT/ethanol_train_PBE_hessian.pkl','rb') as handle:
    hessian_data_train = pickle.load(handle)
len(hessian_data_train)

1000

In [4]:
def generate_training_data_hessian(mol_entries, output_path):
    configs: List[ase.Atoms] = []

    for mol_entry in tqdm(mol_entries):
        crystal_structure: ase.Atoms = mol_entry["atoms"]
        crystal_structure.set_pbc([0,0,0])

        ifc_hessian_matrix = mol_entry['hessian']  # units: eV / Å^2

        print(ifc_hessian_matrix.shape)
        for ind1 in range(ifc_hessian_matrix.shape[0]):
            for ind1c in range(3):
                for ind2 in range(ifc_hessian_matrix.shape[1]):
                    for ind2c in range(3):
                        atoms = crystal_structure.copy()
                        atoms.info["k_point"] = np.array([0.0,0.0,0.0])
                        atoms.info["ifc"] = np.array([ind1,ind1c,ind2,ind2c])
                        atoms.info["ifc_hessian"] = ifc_hessian_matrix[ind1,ind2,ind1c,ind2c]
                        configs.append(atoms)
    print('total number of configs: ',len(configs))

    ase.io.write(output_path, configs, format="extxyz")

In [5]:
generate_training_data_hessian(hessian_data_train[:10], "hessian_test_output.xyz")

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 59.31it/s]


(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
(9, 9, 3, 3)
total number of configs:  7290
